In [6]:
import os
import gc
import sys

## 데이터 관련 라이브러리 로드 

import pandas as pd 
import numpy as np
import re
from tqdm import tqdm

from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets
## LLM, 딥러닝  관련 라이브러리 로드 

import torch 

from transformers import AutoTokenizer #토크나이저 
from transformers import LlamaForCausalLM,  AutoModelForCausalLM
 # LLM 모델 
from transformers import BitsAndBytesConfig # 양자화 라이브러리 
from transformers import GenerationConfig
from transformers import DataCollatorForLanguageModeling

from peft import PeftModel
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training # 효율적 학습을 위한 라이브러리 , LORA 관련 라이브러리 
from transformers import Trainer, TrainingArguments # 학습 관련된 모델 


load dataset

In [2]:
#load dataset : maywell/ko_wikidata_QA
dataset = load_dataset('maywell/ko_wikidata_QA')
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 137505
    })
})

load tokenizer

In [19]:
base_model = "beomi/llama-2-ko-7b"
tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side = 'right')

In [4]:
dataset_prompt = """ ###

### %s 

### %s 

"""

In [5]:
def gen_prompt(element):
    return DatasetDict({'tmp_promt': dataset_prompt%(element['instruction'], element['output'])})


dataset['train'] = dataset['train'].map(gen_prompt)

In [6]:
dataset['train'][0]

{'instruction': '캐모마일은 어떤 식물인가요?',
 'output': "캐모마일은 국화과에 속하는 식물로, 저먼캐모마일, 캐모마일, 카미레 등이 포함됩니다. 이들 식물은 사과와 같은 향이 나기 때문에 '캐모마일'이라고 불립니다. 캐모마일은 노란색 꽃을 가진 식물이며, 6월 중순에서 7월 하순까지 꽃을 피워요. 추위에 강한 편이며, 양지에서 잘 자라요. 꽃은 낮에는 피고 밤에는 오므라져 있으며, 일주일 정도 꽃이 핀다고 합니다. 씨받이는 매우 쉬우며, 싹도 잘 나기 때문에 직접 씨를 받을 수 있습니다. 캐모마일은 양지 바르고 배수가 좋은 사질토에서 잘 피워요. 수확한 꽃은 통풍이 잘 되는 응달에서 말려 저장해요. 캐모마일의 방향유는 긴장 완화, 두통 등의 통증 완화에 효과가 있으며, 몸을 따뜻하게 해주는 효과도 있습니다. 차로 마시면 위장장애의 완화에 도움을 줄 수 있습니다. 캐모마일은 카페인 성분이 없는 허브이며, 좀벌레를 없애는 데 효과적인 허브식물입니다.",
 'tmp_promt': " ###\n\n### 캐모마일은 어떤 식물인가요? \n\n### 캐모마일은 국화과에 속하는 식물로, 저먼캐모마일, 캐모마일, 카미레 등이 포함됩니다. 이들 식물은 사과와 같은 향이 나기 때문에 '캐모마일'이라고 불립니다. 캐모마일은 노란색 꽃을 가진 식물이며, 6월 중순에서 7월 하순까지 꽃을 피워요. 추위에 강한 편이며, 양지에서 잘 자라요. 꽃은 낮에는 피고 밤에는 오므라져 있으며, 일주일 정도 꽃이 핀다고 합니다. 씨받이는 매우 쉬우며, 싹도 잘 나기 때문에 직접 씨를 받을 수 있습니다. 캐모마일은 양지 바르고 배수가 좋은 사질토에서 잘 피워요. 수확한 꽃은 통풍이 잘 되는 응달에서 말려 저장해요. 캐모마일의 방향유는 긴장 완화, 두통 등의 통증 완화에 효과가 있으며, 몸을 따뜻하게 해주는 효과도 있습니다. 차로 마시면 위장장애의 완화에 도움을 줄 수 있습니다. 캐모마일은 카페인 성분이 없는 허브이며, 좀벌레를 없애는 데 효과적인 허브식물입니다. \n\n"}

In [7]:
def tokenize(element):
    
    outputs = tokenizer(
        element['tmp_promt'],
        truncation=True,
        max_length=2048
    )

    return {"input_ids": outputs["input_ids"]}

In [8]:
tokenized_datasets = dataset['train'].map(
    tokenize, batched=True, remove_columns=dataset['train'].column_names
)

Map:   0%|          | 0/137505 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size = 0.2, shuffle =True)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 110004
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 27501
    })
})

load llm model

In [11]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    
mps_device

device(type='mps')

In [9]:
## 4bit quantaziation (양자화 라이브러리가 m1 실리콘에는 지원하지 않기 때문에 양자화된 모델을 다운로드하여 mps 환경에서 훈련합니다.)

bnb_4bit_compute_dtype = "bfloat16"
use_4bit = True

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

In [10]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
compute_dtype

torch.bfloat16

In [14]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.bfloat16 and use_4bit:
    major = torch.mps.driver_allocated_memory()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [2]:
# pytorch_model-00005-of-00005.bin
model = LlamaForCausalLM.from_pretrained(base_model)#, device_map = 'mps')

NameError: name 'LlamaForCausalLM' is not defined

In [49]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)
# model = LlamaForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map = 'mps')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


NameError: name 'bnb_config' is not defined

In [ ]:
args = TrainingArguments(
    output_dir="common_sense_llama",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=1_000,
    logging_steps=1_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=1_000,
    fp16=True,
    push_to_hub=False,
    optim = "adamw_torch",
)

trainer = Trainer(
    accelerator="mps",
    device=1,
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [ ]:
trainer.train()